# **(Feature Engineering)**

## Objectives

* Engineer features for a Regression Model

## Inputs

* outputs/datasets/cleaned/TrainSetCleaned.csv

* outputs/datasets/cleaned/TrainSetCleaned.csv

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

## Load Cleaned Data

- Train Set

In [ ]:
import pandas as pd
train_set_path = "outputs/datasets/cleaned/TrainSetCleaned.csv"
TrainSet = pd.read_csv(train_set_path)
TrainSet.head(3)
print(TrainSet.info())

- Test Set

In [ ]:
test_set_path = 'outputs/datasets/cleaned/TestSetCleaned.csv'
TestSet = pd.read_csv(test_set_path)
TestSet.head(3)

## Data Exploration

In [ ]:
print(set(TrainSet['BsmtExposure'].unique()))
print(set(TrainSet['BsmtFinType1'].unique()))
print(set(TrainSet['GarageFinish'].unique()))
print(set(TrainSet['KitchenQual'].unique()))


---

In [ ]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=TrainSet, minimal=True)
pandas_report.to_notebook_iframe()

## Feature Engineering

- Load the custom function provided for feature engineering

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
from feature_engine import transformation as vt
from feature_engine.outliers import Winsorizer
from feature_engine.encoding import OrdinalEncoder
sns.set(style="whitegrid")
warnings.filterwarnings('ignore')


def FeatureEngineeringAnalysis(df, analysis_type=None):
    """
    - used for quick feature engineering on numerical and categorical variables
    to decide which transformation can better transform the distribution shape
    - Once transformed, use a reporting tool, like pandas-profiling, to evaluate distributions
    """
    check_missing_values(df)
    allowed_types = ['numerical', 'ordinal_encoder', 'outlier_winsorizer']
    check_user_entry_on_analysis_type(analysis_type, allowed_types)
    list_column_transformers = define_list_column_transformers(analysis_type)

    # Loop in each variable and engineer the data according to the analysis type
    df_feat_eng = pd.DataFrame([])
    for column in df.columns:
        # create additional columns (column_method) to apply the methods
        %matplotlib inline
        df_feat_eng = pd.concat([df_feat_eng, df[column]], axis=1)
        for method in list_column_transformers:
            df_feat_eng[f"{column}_{method}"] = df[column]

        # Apply transformers in respective column_transformers
        df_feat_eng, list_applied_transformers = apply_transformers(
            analysis_type, df_feat_eng, column)

        # For each variable, assess how the transformations perform
        transformer_evaluation(
            column, list_applied_transformers, analysis_type, df_feat_eng)

    return df_feat_eng


def check_user_entry_on_analysis_type(analysis_type, allowed_types):
    """ Check analysis type """
    if analysis_type is None:
        raise SystemExit(
            f"You should pass analysis_type parameter as one of the following options: {allowed_types}")
    if analysis_type not in allowed_types:
        raise SystemExit(
            f"analysis_type argument should be one of these options: {allowed_types}")


def check_missing_values(df):
    if df.isna().sum().sum() != 0:
        raise SystemExit(
            f"There is a missing value in your dataset. Please handle that before getting into feature engineering.")


def define_list_column_transformers(analysis_type):
    """ Set suffix columns according to analysis_type"""
    if analysis_type == 'numerical':
        list_column_transformers = [
            "log_e", "log_10", "reciprocal", "power", "box_cox", "yeo_johnson"]

    elif analysis_type == 'ordinal_encoder':
        list_column_transformers = ["ordinal_encoder"]

    elif analysis_type == 'outlier_winsorizer':
        list_column_transformers = ['iqr']

    return list_column_transformers


def apply_transformers(analysis_type, df_feat_eng, column):
    for col in df_feat_eng.select_dtypes(include='category').columns:
        df_feat_eng[col] = df_feat_eng[col].astype('object')

    if analysis_type == 'numerical':
        df_feat_eng, list_applied_transformers = FeatEngineering_Numerical(
            df_feat_eng, column)

    elif analysis_type == 'outlier_winsorizer':
        df_feat_eng, list_applied_transformers = FeatEngineering_OutlierWinsorizer(
            df_feat_eng, column)

    elif analysis_type == 'ordinal_encoder':
        df_feat_eng, list_applied_transformers = FeatEngineering_CategoricalEncoder(
            df_feat_eng, column)

    return df_feat_eng, list_applied_transformers


def transformer_evaluation(column, list_applied_transformers, analysis_type, df_feat_eng):
    # For each variable, assess how the transformations perform
    print(f"* Variable Analyzed: {column}")
    print(f"* Applied transformation: {list_applied_transformers} \n")
    for col in [column] + list_applied_transformers:

        if analysis_type != 'ordinal_encoder':
            DiagnosticPlots_Numerical(df_feat_eng, col)

        else:
            if col == column:
                DiagnosticPlots_Categories(df_feat_eng, col)
            else:
                DiagnosticPlots_Numerical(df_feat_eng, col)

        print("\n")


def DiagnosticPlots_Categories(df_feat_eng, col):
    plt.figure(figsize=(4, 3))
    sns.countplot(data=df_feat_eng, x=col, palette=[
                  '#432371'], order=df_feat_eng[col].value_counts().index)
    plt.xticks(rotation=90)
    plt.suptitle(f"{col}", fontsize=30, y=1.05)
    plt.show()
    print("\n")


def DiagnosticPlots_Numerical(df, variable):
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    sns.histplot(data=df, x=variable, kde=True, element="step", ax=axes[0])
    stats.probplot(df[variable], dist="norm", plot=axes[1])
    sns.boxplot(x=df[variable], ax=axes[2])

    axes[0].set_title('Histogram')
    axes[1].set_title('QQ Plot')
    axes[2].set_title('Boxplot')
    fig.suptitle(f"{variable}", fontsize=30, y=1.05)
    plt.tight_layout()
    plt.show()


def FeatEngineering_CategoricalEncoder(df_feat_eng, column):
    list_methods_worked = []
    try:
        encoder = OrdinalEncoder(encoding_method='arbitrary', variables=[
                                 f"{column}_ordinal_encoder"])
        df_feat_eng = encoder.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_ordinal_encoder")

    except Exception:
        df_feat_eng.drop([f"{column}_ordinal_encoder"], axis=1, inplace=True)

    return df_feat_eng, list_methods_worked


def FeatEngineering_OutlierWinsorizer(df_feat_eng, column):
    list_methods_worked = []

    # Winsorizer iqr
    try:
        disc = Winsorizer(
            capping_method='iqr', tail='both', fold=1.5, variables=[f"{column}_iqr"])
        df_feat_eng = disc.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_iqr")
    except Exception:
        df_feat_eng.drop([f"{column}_iqr"], axis=1, inplace=True)

    return df_feat_eng, list_methods_worked


def FeatEngineering_Numerical(df_feat_eng, column):
    list_methods_worked = []

    # LogTransformer base e
    try:
        lt = vt.LogTransformer(variables=[f"{column}_log_e"])
        df_feat_eng = lt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_log_e")
    except Exception:
        df_feat_eng.drop([f"{column}_log_e"], axis=1, inplace=True)

    # LogTransformer base 10
    try:
        lt = vt.LogTransformer(variables=[f"{column}_log_10"], base='10')
        df_feat_eng = lt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_log_10")
    except Exception:
        df_feat_eng.drop([f"{column}_log_10"], axis=1, inplace=True)

    # ReciprocalTransformer
    try:
        rt = vt.ReciprocalTransformer(variables=[f"{column}_reciprocal"])
        df_feat_eng = rt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_reciprocal")
    except Exception:
        df_feat_eng.drop([f"{column}_reciprocal"], axis=1, inplace=True)

    # PowerTransformer
    try:
        pt = vt.PowerTransformer(variables=[f"{column}_power"])
        df_feat_eng = pt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_power")
    except Exception:
        df_feat_eng.drop([f"{column}_power"], axis=1, inplace=True)

    # BoxCoxTransformer
    try:
        bct = vt.BoxCoxTransformer(variables=[f"{column}_box_cox"])
        df_feat_eng = bct.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_box_cox")
    except Exception:
        df_feat_eng.drop([f"{column}_box_cox"], axis=1, inplace=True)

    # YeoJohnsonTransformer
    try:
        yjt = vt.YeoJohnsonTransformer(variables=[f"{column}_yeo_johnson"])
        df_feat_eng = yjt.fit_transform(df_feat_eng)
        list_methods_worked.append(f"{column}_yeo_johnson")
    except Exception:
        df_feat_eng.drop([f"{column}_yeo_johnson"], axis=1, inplace=True)

    return df_feat_eng, list_methods_worked

## Categorical Encoding

In [ ]:
variables_engineering= ['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual']

variables_engineering

In [ ]:
df_engineering = TrainSet[variables_engineering].copy()
df_engineering.head(3)

In [ ]:
df_engineering = FeatureEngineeringAnalysis(df=df_engineering, analysis_type='ordinal_encoder')

**Note:**

- All transformations were applied correctly.

- Apply the selected transformation to the Train and Test set

In [ ]:
encoder = OrdinalEncoder(encoding_method='arbitrary', variables = variables_engineering)
TrainSet = encoder.fit_transform(TrainSet)
TestSet = encoder.transform(TestSet)

print("* Categorical encoding - ordinal transformation done!")

## Numerical Transformation

In [ ]:
print(TrainSet.columns)

In [ ]:
variables_engineering = ['1stFlrSF', '2ndFlrSF', 'BedroomAbvGr','BsmtFinSF1','BsmtUnfSF', 'GarageArea',
                        'GarageYrBlt', 'GrLivArea','LotArea', 'LotFrontage','MasVnrArea', 'OpenPorchSF', 
                        'OverallCond', 'OverallQual','TotalBsmtSF', 'YearBuilt', 'YearRemodAdd']
variables_engineering

In [ ]:
df_engineering = TrainSet[variables_engineering].copy()
df_engineering.head(3)

In [ ]:
df_engineering = FeatureEngineeringAnalysis(df=df_engineering, analysis_type='numerical')

### Some conclusions from above:

- 1stFloorSF:

    - Applied transformers that attributed to this variables distribution becoming more normally distributed.

        - '1stFlrSF_log_e', '1stFlrSF_log_10', '1stFlrSF_box_cox', '1stFlrSF_yeo_johnson'
        - I Will apply the variance stabilizing transformer: LogTransformer

- 2ndFloorSF:

    - No effective transformer in this category. Too many instances in the set with no 2nd Floor.

- BedroomAbvGr:

    - Applied transformers that attributed to this variables distribution becoming more normally distributed.
    - Note: It is not normal, but there is a very slight improvement.

        - BedroomAbvGr_power

- BsmtFinSF1:

    - No effective transformer in this category.

- BsmtUnfSF'

    - Applied transformers that attributed to this variables distribution becoming more normally distributed.
        
        - 'BsmtUnfSF_power', 'BsmtUnfSF_yeo_johnson'

- GarageArea'

    - Applied transformers that attributed to this variables distribution becoming more normally distributed.
    - Note: Looking at the 0 values here, they provide no value and should be dropped
        
        - 'GarageArea_power'

- GarageYrBlt

    - No effective transformer in this category.

- GrLivArea

    - Applied transformers that attributed to this variables distribution becoming more normally distributed.
        
        - 'GarageArea_Log_e', 'GarageArea_Log_10', 'GarageArea_box_cox', 'GarageArea_yeo_johnson'
        - I Will apply the variance stabilizing transformer: LogTransformer

- LotArea

    - Applied transformers that attributed to this variables distribution becoming more normally distributed.
        
        - LotArea_log_e', LotArea_log_10'
        - I Will apply the variance stabilizing transformer: LogTransformer

- LotFrontage

    - Applied transformers that attributed to this variables distribution becoming more normally distributed.
        
        - 'LotFrontage_log_e', 'LotFrontage_Log10', 'LotFrontage_box_cox', 'LotFrontage_yeo_johnson'
        - I Will apply the variance stabilizing transformer: LogTransformer

- MasVnrArea

    - No effective transformer in this category.

- OpenPorchSF

    - No effective transformer in this category.

- OverallCond

    - No effective transformer in this category.

- OverallQual

    - No effective transformer in this category.(Quite normal in current state)

- TotalBsmtSF

    - No effective transformer in this category.

- YearBuilt

    - No effective transformer in this category.

- YearRemodAdd

    - No effective transformer in this category.



In [ ]:
variables_engineering = ['1stFlrSF', 'GrLivArea', 'LotArea', 'LotFrontage']

In [ ]:
from feature_engine.transformation import LogTransformer

log_10_transformer = LogTransformer(variables=variables_engineering, base='10')
TrainSet = log_10_transformer.fit_transform(TrainSet)
TestSet = log_10_transformer.transform(TestSet)

print(" Variance stabilizing transformer 'Log10' applied to '1stFlrSF', 'GrLivArea', 'LotArea', 'LotFrontage'!")

In [ ]:
variables_engineering = ['BedroomAbvGr', 'BsmtUnfSF', 'GarageArea']

In [ ]:
from feature_engine.transformation import PowerTransformer

power_transformer = PowerTransformer(variables=variables_engineering, exp=0.5)
TrainSet = power_transformer.fit_transform(TrainSet)
TestSet = power_transformer.transform(TestSet)

print(" Variance stabilizing transformer 'power' applied to 'BedroomAbvGr', 'BsmtUnfSF', 'GarageArea'!")

In [ ]:
# Missed the TotalBsmtSF category in the previous transformer
variables_engineering = ['TotalBsmtSF']
variables_engineering

In [ ]:
power_transformer = PowerTransformer(variables=variables_engineering, exp=0.5)
TrainSet = power_transformer.fit_transform(TrainSet)
TestSet = power_transformer.transform(TestSet)
print(" Variance stabilizing transformer 'power' applied to 'TotalBsmtSF'!")

In [ ]:
variables_engineering = ['LotArea', 'GrLivArea', 'LotFrontage', 'TotalBsmtSF']
variables_engineering

In [ ]:
from feature_engine.outliers import OutlierTrimmer

outlier_transformer = OutlierTrimmer(variables=variables_engineering , capping_method='gaussian',tail='both')

TrainSet = outlier_transformer.fit_transform(TrainSet)
TestSet = outlier_transformer.transform(TestSet)

print("Applied outlier handler transformer Outlier Trimmer to 'LotArea', 'GrLivArea', 'LotFrontage', 'TotalBsmtSF'!")

In [ ]:
# Copy the entire Train Set to apply smart correlation selection
df_engineering = TrainSet.copy()
df_engineering.head(3)

In [ ]:
from feature_engine.selection import SmartCorrelatedSelection

variables_correlation = SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.7, selection_method="variance")

variables_correlation.fit_transform(df_engineering)
variables_correlation.correlated_feature_sets_

In [ ]:
variables_correlation.features_to_drop_

## Pearson Results:

- 0.7 Threshold pearsons:['1stFlrSF', 'GarageYrBlt', 'OverallQual']

- 0.6 Threshold pearsons: ['1stFlrSF', 'GarageArea', 'GarageYrBlt', 'GrLivArea', 'OverallQual']

## Spearman Results:

- 0.7 Threshold spearmans: ['1stFlrSF', 'GarageYrBlt', 'GrLivArea', 'YearRemodAdd']

- 0.6 Threshold spearmans: ['1stFlrSF','BsmtUnfSF','GarageArea','GarageYrBlt', 'GrLivArea',
 'OverallQual','YearRemodAdd']

# Conclusions for ML Pipline requirements:

- Ordinal encoding required for:
    - 'BsmtExposure'
    - 'BsmtFinType1'
    - 'GarageFinish'
    - 'KitchenQual'

- Log Transformer required for:
    - '1stFlrSF'
    - 'GrLivArea'
    - 'LotArea'
    - 'LotFrontage'

- Power transformer required for:
    - 'BedroomAbvGr'
    - 'BsmtUnfSF'
    - 'GarageArea'
    - TotalBsmtSF

- Outlier Trimmer required for:
    - 'LotArea'
    - 'GrLivArea'
    - 'LotFrontage'
    - 'TotalBsmtSF'

- 0.7 threshold from pearson smart correlation selection results:
    - '1stFlrSF'
    - 'GarageYrBlt'
    - 'OverallQuall'

- Drop these categories which were 90% empty values:
    - 'EnclosedPorch', 'WoodDeckSF'

---